<a href="https://colab.research.google.com/github/Amanda9805/Detecting-Machine-Generated-Texts/blob/train-model/COS_760_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --upgrade transformers datasets fsspec evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 36.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 26.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 5.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: transformers
    Found existing installation: transformers 4.52.2
    Uninstalling transformers-4.52.2:
      Successfully uninstalled transformers-4.52.2
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcs

In [2]:
import pandas as pd
import numpy as np
import re
import nltk
import evaluate
import torch

from huggingface_hub import login
from datasets import load_dataset, Dataset, ClassLabel
from transformers import (AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, DataCollatorWithPadding, EarlyStoppingCallback)
from sklearn.model_selection import train_test_split
from nltk.tokenize import word_tokenize

nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [3]:
def clean_text(text):
    # Remove URLs
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    # Remove special characters and numbers
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    # Convert to lowercase
    text = text.lower()
    # Remove extra whitespace
    text = ' '.join(text.split())
    return text

In [4]:
#shona_data = load_dataset("DigitalUmuganda/AfriVoice", "sn", streaming=True, split="train[:10%]")

# english_data = load_dataset("oscar-corpus/OSCAR-2201", language="en", streaming=True)

# num_samples = 500
# samples = []
# for i, example in enumerate(english_data["train"]):
#     if i >= num_samples:
#         break
#     samples.append(example['text'])

# eng_df = pd.DataFrame(samples, columns=['text'])
# eng_df.head()

zulu_data = load_dataset("dsfsi/vukuzenzele-monolingual", "zul")

def reformatData(dataDict):
  langList = []
  for index, row in dataDict.iterrows():
    # Access data using column names from the row Series
    if 'text' in row and row['text']:
        langList.append({
        'text': row.get('text', ''),
        'author': row.get('author', ''),
        'title': row.get('title', ''),
        'language': 'zul'
              })

  return langList

human_data = reformatData(zulu_data['train'].to_pandas())

zul_df = pd.DataFrame(human_data)
zul_df['cleaned_text'] = zul_df['text'].apply(clean_text)
zul_df['tokens'] = zul_df['cleaned_text'].apply(word_tokenize)
zul_df['label'] = 0
zul_df.head()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/11.2k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/312k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/73.8k [00:00<?, ?B/s]

eval-00000-of-00001.parquet:   0%|          | 0.00/74.0k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/129 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/28 [00:00<?, ? examples/s]

Generating eval split:   0%|          | 0/28 [00:00<?, ? examples/s]

,text,author,title,language,cleaned_text,tokens,label
0,Ngokusho kukaNgqongqoshe Wezempilo uDkt. Joe P...,Allison Cooper\n,Zivikele kwi-COVID-19 ngaphambi kobusika \n,zul,ngokusho kukangqongqoshe wezempilo udkt joe ph...,"[ngokusho, kukangqongqoshe, wezempilo, udkt, j...",0
1,Abafundi BakaMa tikuletsheni bakulonyaka kumel...,Dale Hes \n,Kwethulwe ngokusemthethweni uhlelo lweWoza Ma...,zul,abafundi bakama tikuletsheni bakulonyaka kumel...,"[abafundi, bakama, tikuletsheni, bakulonyaka, ...",0
2,Lolu hlelo luhlose ukuthuthukisa ikhono labant...,Vukuzenzele Unnamed\n,Uhlelo lokudla esikoleni luqinisekisa ukuthi a...,zul,lolu hlelo luhlose ukuthuthukisa ikhono labant...,"[lolu, hlelo, luhlose, ukuthuthukisa, ikhono, ...",0
3,hulumeni eNingizimu Afrika yonkana abakwazi uk...,More Matshediso\n,Izisu ezigcwele ziholela emiphumeleni engcono\n,zul,hulumeni eningizimu afrika yonkana abakwazi uk...,"[hulumeni, eningizimu, afrika, yonkana, abakwa...",0
4,Kamuva nje bengivakashele eThekwini KwaZulu-Na...,Vukuzenzele Unnamed\n,Ukwakha kabusha impilo nendlela yokuphila emuv...,zul,kamuva nje bengivakashele ethekwini kwazulunat...,"[kamuva, nje, bengivakashele, ethekwini, kwazu...",0


In [5]:
# eng_df['cleaned_text'] = eng_df['text'].apply(clean_text)
# eng_df['tokens'] = eng_df['cleaned_text'].apply(word_tokenize)
# print(eng_df.head())

zul_df['cleaned_text'] = zul_df['text'].apply(clean_text)
zul_df['tokens'] = zul_df['cleaned_text'].apply(word_tokenize)
zul_df.head()

,text,author,title,language,cleaned_text,tokens,label
0,Ngokusho kukaNgqongqoshe Wezempilo uDkt. Joe P...,Allison Cooper\n,Zivikele kwi-COVID-19 ngaphambi kobusika \n,zul,ngokusho kukangqongqoshe wezempilo udkt joe ph...,"[ngokusho, kukangqongqoshe, wezempilo, udkt, j...",0
1,Abafundi BakaMa tikuletsheni bakulonyaka kumel...,Dale Hes \n,Kwethulwe ngokusemthethweni uhlelo lweWoza Ma...,zul,abafundi bakama tikuletsheni bakulonyaka kumel...,"[abafundi, bakama, tikuletsheni, bakulonyaka, ...",0
2,Lolu hlelo luhlose ukuthuthukisa ikhono labant...,Vukuzenzele Unnamed\n,Uhlelo lokudla esikoleni luqinisekisa ukuthi a...,zul,lolu hlelo luhlose ukuthuthukisa ikhono labant...,"[lolu, hlelo, luhlose, ukuthuthukisa, ikhono, ...",0
3,hulumeni eNingizimu Afrika yonkana abakwazi uk...,More Matshediso\n,Izisu ezigcwele ziholela emiphumeleni engcono\n,zul,hulumeni eningizimu afrika yonkana abakwazi uk...,"[hulumeni, eningizimu, afrika, yonkana, abakwa...",0
4,Kamuva nje bengivakashele eThekwini KwaZulu-Na...,Vukuzenzele Unnamed\n,Ukwakha kabusha impilo nendlela yokuphila emuv...,zul,kamuva nje bengivakashele ethekwini kwazulunat...,"[kamuva, nje, bengivakashele, ethekwini, kwazu...",0


In [7]:
from transformers import pipeline
import json

en_generator = pipeline('text-generation', model='gpt2')


english_prompts = [
    "Explain the significance of lobola in Southern Africa",
    "Write a short dialogue between two friends in Johannesburg",
    "Describe linguistic features that make isiZulu agglutinative"
]

zulu_prompts = [
    "Chaza ngokubaluleka kwesiko lwelobola eNingizimu Afrika",
    "Bhala inkulumo emfushane phakathi kwabangani ababili eGoli",
    "Landela indaba yamaZulu ngokomlando",
    "Chaza ngamasiko amasha eZulu eskhathini samanje",
    "Bhala inganekwane ethi 'UNogwaja noFudu'"
]

# def generate_with_prompts(prompts, generator, language, samples_per_prompt=3):
#     data = []
#     for prompt in prompts:
#         for _ in range(samples_per_prompt):
#             output = generator(prompt, max_length=100, do_sample=True, temperature=0.7)
#             data.append({
#                 'prompt': prompt,
#                 'text': output[0]['generated_text'],
#                 'label': 'machine',
#                 'language': language,
#                 'prompt_type': 'cultural' if "tsika" in prompt else 'linguistic'  # Tag for analysis
#             })
#     return pd.DataFrame(data)


# eng_mg_df = generate_with_prompts(english_prompts, en_generator, 'English')
# eng_mg_df['cleaned_text'] = eng_mg_df['text'].apply(clean_text)
# eng_mg_df['tokens'] = eng_mg_df['cleaned_text'].apply(word_tokenize)
# print(eng_mg_df['text'].iloc[0][:300])

def load_jsonl_data(file_path):
    data = []
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            for line in file:
                line = line.strip()
                if line:  # Skip empty lines
                    try:
                        data.append(json.loads(line))
                    except json.JSONDecodeError as e:
                        print(f"Error parsing JSON line: {e}")
                        continue
    except FileNotFoundError:
        print(f"File not found: {file_path}")
        return []

    return data

zul_mg_df = pd.DataFrame(load_jsonl_data('zulu_mg_text.json'))

machine_data =reformatData(zul_mg_df)
zul_mg_df = pd.DataFrame(machine_data)
zul_mg_df['text'] = zul_mg_df['text'].apply(lambda x: ' '.join(x) if isinstance(x, list) else x)
zul_mg_df['language'] = 'zul'
zul_mg_df['cleaned_text'] = zul_mg_df['text'].apply(clean_text)
zul_mg_df['tokens'] = zul_mg_df['cleaned_text'].apply(word_tokenize)
zul_mg_df['label'] = 1

zul_mg_df.head()

Device set to use cuda:0


,text,author,title,language,cleaned_text,tokens,label
0,Uhulumeni uzivezile izinhlelo ezintsha zokwakh...,Sipho Mthembu,Uhulumeni Uvula Uhlelo Olusha Lwezindlu,zul,uhulumeni uzivezile izinhlelo ezintsha zokwakh...,"[uhulumeni, uzivezile, izinhlelo, ezintsha, zo...",1
1,Izinkulungwane zabantu ziye zahlangana namuhla...,Nomsa Dlamini,Abantu Bayaqhubeka Ukusola Ngamanani Kaphethiloli,zul,izinkulungwane zabantu ziye zahlangana namuhla...,"[izinkulungwane, zabantu, ziye, zahlangana, na...",1
2,Ithimba leKaizer Chiefs lifinyelele kokugcina ...,Mandla Zulu,Ithimba LeKaizer Chiefs Lingene Kokugcina KweN...,zul,ithimba lekaizer chiefs lifinyelele kokugcina ...,"[ithimba, lekaizer, chiefs, lifinyelele, kokug...",1
3,UMnyango Wezempilo ufuna ukukhumbuza abantu ng...,Dr. Zanele Ngcobo,Ukubaluleka Kokujikelezwa Kwesifo SeCovid-19,zul,umnyango wezempilo ufuna ukukhumbuza abantu ng...,"[umnyango, wezempilo, ufuna, ukukhumbuza, aban...",1
4,Ucwaningo olusha luveza ukuthi ukuntuleka kwam...,Prof. Nomthandazo Cele,Ukutholakala Kwamanzi Ahlanzekile Kuyisihloko ...,zul,ucwaningo olusha luveza ukuthi ukuntuleka kwam...,"[ucwaningo, olusha, luveza, ukuthi, ukuntuleka...",1


COMBINE THE DATASETS
-------------------------

In [8]:
# Combine both datasets, this will be bad for us if we have more than 70% differnce in data length
# Combine both datasets
all_texts = pd.concat([zul_df, zul_mg_df], ignore_index=True)
all_texts_shuffled = all_texts.sample(frac=1).reset_index(drop=True)
print(f"Total texts after combining: {len(all_texts)}")
print(f"Human-written texts: {len(zul_df)}")
print(f"Machine-generated texts: {len(zul_mg_df)}")

# Create a DataFrame for easier analysis
combined_df = pd.DataFrame(all_texts_shuffled)
combined_df.head()

Total texts after combining: 166
Human-written texts: 126
Machine-generated texts: 40


,text,author,title,language,cleaned_text,tokens,label
0,"Lesi sikole, esizinze eMthatha eMpumalanga Kap...",More Matshediso \n,I-ASIDI yakha ngokushesha ingqalasizinda yesik...,zul,lesi sikole esizinze emthatha empumalanga kapa...,"[lesi, sikole, esizinze, emthatha, empumalanga...",0
1,Sekumba eqolo ukuthenga ukudla kanye nezinye i...,\n,Ukunyuka kwezindleko zokuphila kubiza ukuthi s...,zul,sekumba eqolo ukuthenga ukudla kanye nezinye i...,"[sekumba, eqolo, ukuthenga, ukudla, kanye, nez...",0
2,I-Kevali Chemicals iyinka mpani yabantu abamny...,More Matshediso\n,Osozimboni abamnyama abenza amakhemikhali base...,zul,ikevali chemicals iyinka mpani yabantu abamnya...,"[ikevali, chemicals, iyinka, mpani, yabantu, a...",0
3,"Abantu abahle, qaphelani! Intengo yokudla iyak...",Phindile Mthethwa,Intengo Yokudla Iyakhuphuka,zul,abantu abahle qaphelani intengo yokudla iyakhu...,"[abantu, abahle, qaphelani, intengo, yokudla, ...",1
4,Umgudu kawoyela olawulwa yinkampani i-BP South...,Vukuzenzele Unnamed\n,Ukuthuthukiswa Komnotho Wabansundu kuyimpoqo e...,zul,umgudu kawoyela olawulwa yinkampani ibp southe...,"[umgudu, kawoyela, olawulwa, yinkampani, ibp, ...",0


In [9]:
# Create a balanced dataset with equal samples from each class
# we can choose to use this of the combined one
def create_balanced_dataset(df, target_size_per_class=None):
    class_counts = df['label'].value_counts()
    min_class_size = class_counts.min()
    if target_size_per_class:
        sample_size = min(target_size_per_class, min_class_size)
    else:
        sample_size = min_class_size
    balanced_df = df.groupby('label').sample(n=sample_size, random_state=42)
    return balanced_df.reset_index(drop=True)

# Create balanced dataset, because we need the same number of samples for each class
balanced_df = create_balanced_dataset(combined_df)
print(f"\nBalanced dataset created with {len(balanced_df)} samples")
print(f"Label distribution in balanced dataset: \n{balanced_df['label'].value_counts()}")

# Save balanced dataset
balanced_output_path = 'balanced_zulu_texts.csv'
balanced_df.to_csv(balanced_output_path, index=False, encoding='utf-8')
print(f"Balanced dataset saved to: {balanced_output_path}")
balanced_df.head()


Balanced dataset created with 80 samples
Label distribution in balanced dataset: 
label
0    40
1    40
Name: count, dtype: int64
Balanced dataset saved to: balanced_zulu_texts.csv


,text,author,title,language,cleaned_text,tokens,label
0,"“Aninodwa kulokho,” kusho ungcweti we-zokusebe...",Vukuzenzele unnamed\n,Ukubhekana nokushonelwa ngothandiweyo\n,zul,aninodwa kulokho kusho ungcweti wezokusebenza ...,"[aninodwa, kulokho, kusho, ungcweti, wezokuseb...",0
1,Sasungula uhlelo loku siza abantu ngomse benzi...,vukuzenzele unnamed\n,UHlelo Loxhaso Lwezemisebenzi LukaMongameli lu...,zul,sasungula uhlelo loku siza abantu ngomse benzi...,"[sasungula, uhlelo, loku, siza, abantu, ngomse...",0
2,Abafundi balokhu bebhekene ne zinselele ezini...,More Matshediso\n,Siza abafundi ukuthi babhekane neCOVID-19\n,zul,abafundi balokhu bebhekene ne zinselele ezinin...,"[abafundi, balokhu, bebhekene, ne, zinselele, ...",0
3,"Umbiko, ohlanganisa nekota yokugcina yowezi-20...",More Matshediso\n,Sindisani obhejane bakithi\n,zul,umbiko ohlanganisa nekota yokugcina yowezi ukh...,"[umbiko, ohlanganisa, nekota, yokugcina, yowez...",0
4,Noma ngabe bahlala kuphi futhi noma ngabe benz...,vukuzenzele unnamed\n,Intsha yaseNingizimu Afrika ingumthombo wogqoz...,zul,noma ngabe bahlala kuphi futhi noma ngabe benz...,"[noma, ngabe, bahlala, kuphi, futhi, noma, nga...",0


Model training section
----------------------

In [10]:
# Set random seed for reproducibility
torch.manual_seed(42)
np.random.seed(42)

# Load the balanced dataset
balanced_data_path = 'balanced_zulu_texts.csv'
df = pd.read_csv(balanced_data_path, encoding='utf-8')

# Convert pandas DataFrame to Hugging Face Dataset
dataset = Dataset.from_pandas(df[['text', 'label']])

# Cast the 'label' column to ClassLabel
dataset = dataset.cast_column('label', ClassLabel(num_classes=2, names=['human', 'machine']))

# Split into train and validation sets
train_val_split = dataset.train_test_split(test_size=0.2, seed=42, stratify_by_column='label')
train_dataset = train_val_split['train']
val_dataset = train_val_split['test']

# Load AfroXLMR tokenizer
model_name = "Davlan/afro-xlmr-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Tokenization function
def tokenize_function(examples):
    return tokenizer(
        examples['text'],
        padding="max_length",
        truncation=True,
        max_length=512
    )

# Tokenize datasets
train_dataset = train_dataset.map(tokenize_function, batched=True)
val_dataset = val_dataset.map(tokenize_function, batched=True)

# Set format for PyTorch
train_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])
val_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])

# Load AfroXLMR model for sequence classification
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=2,
    ignore_mismatched_sizes=True
)

# Define metrics for evaluation
accuracy_metric = evaluate.load("accuracy")
f1_metric = evaluate.load("f1")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    accuracy = accuracy_metric.compute(predictions=predictions, references=labels)
    f1 = f1_metric.compute(predictions=predictions, references=labels, average='weighted')
    return {
        'accuracy': accuracy['accuracy'],
        'f1': f1['f1']
    }

# Define training arguments
training_args = TrainingArguments(
    output_dir='/afroxlmr_finetuned',
    run_name='/afroxlmr_finetune_zulu',  # too much warnings so better to use a custom run name
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=50,
    weight_decay=0.01,
    logging_dir='logs',
    logging_steps=10,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    greater_is_better=True,
    fp16=torch.cuda.is_available(),  # Enable mixed precision only if GPU is available
    report_to="none"
)

# Initialize data collator
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# Initialize trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
)

# Fine-tune the model
trainer.train()

# Save the fine-tuned model and tokenizer
model.save_pretrained('/afroxlmr_finetuned/model')
tokenizer.save_pretrained('/afroxlmr_finetuned/tokenizer')

# Evaluate the model
eval_results = trainer.evaluate()
print(f"Evaluation results: {eval_results}")

Casting the dataset:   0%|          | 0/80 [00:00<?, ? examples/s]

tokenizer_config.json:   0%|          | 0.00/398 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/707 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at Davlan/afro-xlmr-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,0.682434,0.625000,0.619048
2,0.701600,0.626953,0.937500,0.937255
3,0.651900,0.381363,1.000000,1.000000


Evaluation results: {'eval_loss': 0.3813629150390625, 'eval_accuracy': 1.0, 'eval_f1': 1.0, 'eval_runtime': 0.1977, 'eval_samples_per_second': 80.92, 'eval_steps_per_second': 10.115, 'epoch': 3.0}
